In [1]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# Dados

In [2]:
planilha = 'Data for capacitated vehicle routing problem.xlsx'

# Origem
df_origem = pd.read_excel(planilha, sheet_name = 'Origem')
origem = df_origem.get(['Latitude', 'Longitude']).values.tolist()[0]
# print(origem)

# Custos dos caixeiros
df_veiculos = pd.read_excel(planilha, sheet_name = 'Veículos')
custos_caixeiros = dict(df_veiculos.get(['Placa', 'Custo / Km']).values)
custos_caixeiros = {str(c): float(custos_caixeiros[c]) for c in custos_caixeiros}
# print(custos_caixeiros)

# Volumes dos caixeiros
volumes_caixeiros = dict(df_veiculos.get(['Placa', 'Cubagem (m³)']).values)
volumes_caixeiros = {str(c): float(volumes_caixeiros[c]) for c in volumes_caixeiros}
# print(volumes_caixeiros)

# Pesos dos caixeiros
pesos_caixeiros = dict(df_veiculos.get(['Placa', 'Lotação (kg)']).values)
pesos_caixeiros = {str(c): float(pesos_caixeiros[c]) for c in pesos_caixeiros}
# print(pesos_caixeiros)

# Pontos de entrega
df_pedidos = pd.read_excel(planilha, sheet_name = 'Pedido')
entregas = sorted(list(set(df_pedidos.get('Destinatário'))))[:50]
pontos = pd.read_excel(planilha, sheet_name = 'Destinatário', \
                       usecols = ['Código', 'Latitude', 'Longitude']).values.tolist()
pontos_entregas = [[float(l1), float(l2)] for c, l1, l2 in pontos if c in entregas]
# print(pontos_entregas)

# Volumes e pesos das entregas
df_produtos = pd.read_excel(planilha, sheet_name = 'Produto')

volumes_entregas = []
pesos_entregas = []
for e in entregas:
    volume = float(0)
    peso = float(0)
    for p in df_pedidos.loc[df_pedidos['Destinatário'] == e].values.tolist():
        prod = p[1]
        quan = float(p[3])
        corr = df_produtos.loc[df_produtos['Código'] == prod].values.tolist()
        volume += quan * float(corr[0][4])
        peso += quan * float(corr[0][3])
    volumes_entregas.append(volume)
    pesos_entregas.append(peso)

# Função objetivo

O objetivo principal dessa seção é definir a função objetivo do nosso problema.

Para começar, vamos definir uma função que, a partir de uma lista de vetores em $\mathbb R^n$, constrói uma matriz cujas entradas são as distâncias entre os vetores da lista dada.

In [ ]:
# Dada uma lista de vetores em Rn, essa função retorna uma matriz contendo
# as distancias euclidianas entre os vetores da lista.

def matriz_distancia(lista_arrays):
    n = len(lista_arrays)
    matriz = np.zeros((n,n))
    for i in range(n):
        matriz[i,i] = np.inf
        for j in range(i):
            matriz[i,j] = np.linalg.norm(np.array(lista_arrays[i]) - np.array(lista_arrays[j]))
            matriz[j,i] = matriz[i,j]
    return matriz

Em seguida, nós vamos definir uma função que, dada uma trajetória (uma lista
de vetores em $\mathbb R^n$) e o custo para um veículo percorrer um quilômetro
(um número), retorna o custo total desse veículo percorrer toda a trajetória.

In [ ]:
# Dada uma lista de arrays (trajetoria do caixeiro) e um numero (custo do 
# uso do caixeiro por km), retorna a FO desse caixeiro, ou seja, o custo 
# total do caixeiro percorrer a trajetoria dada na lista.

def fo_caixeiro(lista_arrays, numero):
    trajetoria = lista_arrays
    distancia = matriz_distancia(trajetoria)
    custo_caixeiro = numero
    
    fo = 0
    for i in range(len(trajetoria)-1):
        fo += distancia[i][i+1]
    fo += distancia[len(trajetoria)-1][0]
    fo *= custo_caixeiro
    return fo

Para terminar essa seção, vamos definir uma função que retorna a FO do nosso problema.
As entradas dessa função são: um dicionário de listas de arrays, onde cada entrada
representa um caixeiro (string) e a trajetória que ele irá percorrer (lista de vetores
em $\mathbb R^n$); e um dicionário de números, onde cada entrada representa um caixeiro 
(string) e o custo desse caixeiro percorrer um quilômetro.

In [ ]:
# Dados dois dicionários, um dicionário de listas de arrays (trajetorias de 
# cada veículo) e um dicionário de números (custos por km de cada veiculo),
# retorna a FO, ou seja, a soma dos custos de todos veiculos percorrerem os
# pontos das suas respectivas trajetorias.

def fo(dicionario_listas_arrays, dicionario_numeros):
    trajetorias = dicionario_listas_arrays
    custos = dicionario_numeros
    
    caixeiros = trajetorias.keys()
    
    fo = 0
    for caixeiro in caixeiros:
        fo += fo_caixeiro(trajetorias[caixeiro], custos[caixeiro])
    return fo

## Algumas funções auxiliares

Nessa seção, vamos definir duas funções auxiliares.

A primeira ajuda na vizualização dos resultados, plotando as trajetórias de cada caixeiro.

In [ ]:
# Uma função para ajudar na vizualização dos resultados.  Dados um array (um
# vetor em Rn, que representa a origem dos caixeiros) e um dicionário de 
# listas de arrays (que representa as trajetórias dos caixeiros), essa 
# função plota cada uma dessas trajetorias com uma cor diferente.

def ver_rotas(array, dicionario_listas_arrays):
    origem = array
    caixeiros = dicionario_listas_arrays.keys()
    trajetorias = [[*dicionario_listas_arrays[c], origem] for c in caixeiros]
        
    fig = plt.figure()
    axes = fig.add_subplot()
    
    xmin = min([p[0] for t in trajetorias for p in t])-1
    xmax = max([p[0] for t in trajetorias for p in t])+1
    ymin = min([p[1] for t in trajetorias for p in t])-1
    ymax = max([p[1] for t in trajetorias for p in t])+1
    
    axes.set(xlim = (xmin, xmax))
    axes.set(ylim = (ymin, ymax))
    
    xdata = [[p[0] for p in t] for t in trajetorias]
    ydata = [[p[1] for p in t] for t in trajetorias]
    
    for i in range(len(trajetorias)):
        axes.scatter(xdata[i], ydata[i])
        axes.plot(xdata[i], ydata[i])
    axes.plot(origem[0], origem[1], 'o', color = 'black')
    plt.show()

A segunda é uma função que transforma uma lista de números em um dicionário de listas de números.  Ela será usada nas meta-heurísticas.

In [ ]:
# As entradas dessa função são: uma lista de arrays (vetores em R2 que 
# representam todos os pontos de entrega possíveis), uma lista de números
# (que representam os dados de cada um desses pontos de entrega - por
# exemplo, peso e volume) e um dicionário de lista de arrays (cujas entradas
# são caixeiros e as suas respectivas trajetórias).  A saída é um dicionário
# de listas de números, que representam os dados (por exemplo, peso ou 
# volume) relativos a cada ponto de cada trajetória.

def lista2dict(lista_arrays, lista_numeros, dicionario_listas_arrays):
    pontos_entregas = lista_arrays
    dados_entregas = lista_numeros
    caixeiros = dicionario_listas_arrays.keys()
    trajetorias = dicionario_listas_arrays
    
    dict_dados = dict()
    for c in caixeiros:
        dict_dados[c] = [dados_entregas[pontos_entregas.index(p)] if p in pontos_entregas else 0 for p in trajetorias[c]]
    return dict_dados

# Heurísticas

Nessa seção, vamos definir diversas funções que impementam heurísticas usadas para resolver nosso problema principal.

## Heurísticas construtivas

Para começar, vamos implementar duas heurísticas construtivas.  Essas heurísticas constroem soluções iniciais para o nosso problema, que serão melhoradas depois por outras heurísticas (Christofides-Serdyukov e buscas locais).

### Herística construtiva gulosa do vizinho mais próximo

In [ ]:
# Essa heurística constrói trajetorias para cada um dos caixeiros de forma
# que eles façam a maior quantidade possível de entregas.  As entradas dessa
# função são: um array (um vetor de R2 que representa a origem dos
# caixeiros), um dicionário de números (custo por km de cada caixeiro), um
# segundo dicionário de números (volume total que cada caixeiro consegue 
# transportar), um terceiro dicionário de números (peso total que cada
# caixeiro consegue transportar), uma lista de listas de arrays (que
# representa todos os pontos de entregas de produtos - vetores de R2), uma
# lista de números (os volumes de cada uma dos pontos de entrega) e uma 
# segunda lista de números (os pesos de cada um dos pontos de entrega).  A
# saída dessa função é um dicionário de listas de arrays, que representa as
# trajetórias de cada caixeiro.

def construtiva_vizinho(array, dict_nums1, dict_nums2, dict_nums3, lista_vetores, lista_nums1, lista_nums2):
    origem = copy.deepcopy(array)
    custos_caixeiros = copy.deepcopy(dict_nums1)
    volumes_caixeiros = copy.deepcopy(dict_nums2)
    pesos_caixeiros = copy.deepcopy(dict_nums3)
    pontos_entregas = copy.deepcopy(lista_vetores)
    volumes_entregas = copy.deepcopy(lista_nums1)
    pesos_entregas = copy.deepcopy(lista_nums2)

    trajetorias = dict()
    
    while len(pontos_entregas) > 0 and len(custos_caixeiros) > 0:
        caixeiro = min(custos_caixeiros, key = custos_caixeiros.get)
        trajetoria = [origem]
        
        ponto_atual = origem
        volume_atual = volumes_caixeiros[caixeiro]
        peso_atual = pesos_caixeiros[caixeiro]
        
        possiveis_proximos = [(p, v, w) for p in pontos_entregas for v in volumes_entregas for w in pesos_entregas \
                              if v <= volume_atual and w <= peso_atual]
        proxima_entrega = ponto_atual
        volume_entrega = 0
        peso_entrega = 0
        if len(possiveis_proximos) > 0:
            dis_min = min([np.linalg.norm(np.array(ponto_atual) - np.array(p)) for p in possiveis_proximos[0]])
            i = np.argmin([np.linalg.norm(np.array(ponto_atual) - np.array(p)) for p in possiveis_proximos[0]])
            proxima_entrega, volume_entrega, peso_entrega = possiveis_proximos[i]

        while ponto_atual != proxima_entrega and len(pontos_entregas) > 0:
            trajetoria.append(proxima_entrega)
            ponto_atual = proxima_entrega
            volume_atual -= volume_entrega
            peso_atual -= peso_entrega
            
            i = pontos_entregas.index(ponto_atual)
            pontos_entregas.pop(i)
            volumes_entregas.pop(i)
            pesos_entregas.pop(i)

            possiveis_proximos = [(p, v, w) for p in pontos_entregas for v in volumes_entregas for w in pesos_entregas \
                                  if v <= volume_atual and w <= peso_atual]
            proxima_entrega = ponto_atual
            volume_entrega = 0
            peso_entrega = 0
            if len(possiveis_proximos) > 0:
                dis_min = min([np.linalg.norm(np.array(ponto_atual) - np.array(p)) for p in possiveis_proximos[0]])
                i = np.argmin([np.linalg.norm(np.array(ponto_atual) - np.array(p)) for p in possiveis_proximos[0]])
                proxima_entrega, volume_entrega, peso_entrega = possiveis_proximos[i]

        #Descomentar para que a trajetoria seja um ciclo:
        #trajetoria.append(origem)
        custos_caixeiros.pop(caixeiro)
        volumes_caixeiros.pop(caixeiro)
        pesos_caixeiros.pop(caixeiro)
        if len(trajetoria) > 1:
            trajetorias[caixeiro] = trajetoria
        
    return trajetorias

### Heurística construtiva gulosa aleatória (para o GRASP)

In [ ]:
# Essa heurística constrói trajetorias para cada um dos caixeiros de forma
# que eles façam a maior quantidade possível de entregas.  As entradas dessa
# função são: um número (que determina o quão aleatória essa heurística será)
# um array (um vetor de R2 que representa a origem dos caixeiros), um
# dicionário de números (custo por km de cada caixeiro), um segundo
# dicionário de números (volume total que cada caixeiro consegue 
# transportar), um terceiro dicionário de números (peso total que cada
# caixeiro consegue transportar), uma lista de listas de arrays (que
# representa todos os pontos de entregas de produtos - vetores de R2), uma
# lista de números (os volumes de cada uma dos pontos de entrega) e uma 
# segunda lista de números (os pesos de cada um dos pontos de entrega).  A
# saída dessa função é um dicionário de listas de arrays, que representa as
# trajetórias de cada caixeiro.


def construtiva_aleatoria(numero, array, dict_nums1, dict_nums2, dict_nums3, lista_vetores, lista_nums1, lista_nums2):
    alpha = copy.deepcopy(numero)
    origem = copy.deepcopy(array)
    custos_caixeiros = copy.deepcopy(dict_nums1)
    volumes_caixeiros = copy.deepcopy(dict_nums2)
    pesos_caixeiros = copy.deepcopy(dict_nums3)
    pontos_entregas = copy.deepcopy(lista_vetores)
    volumes_entregas = copy.deepcopy(lista_nums1)
    pesos_entregas = copy.deepcopy(lista_nums2)

    trajetorias = dict()
    
    while len(pontos_entregas) > 0 and len(custos_caixeiros) > 0:
        caixeiro = min(custos_caixeiros, key = custos_caixeiros.get)
        trajetoria = [origem]
        
        ponto_atual = origem
        volume_atual = volumes_caixeiros[caixeiro]
        peso_atual = pesos_caixeiros[caixeiro]
        
        proxima_entrega = origem
        vetor_distancia = [np.linalg.norm(np.array(ponto_atual) - np.array(p)) for p in pontos_entregas]
        delta = min(vetor_distancia) + alpha*(max(vetor_distancia) - min(vetor_distancia))
        possiveis_entregas = [(pontos_entregas[i], v, w) \
                              for i in range(len(pontos_entregas)) for v in volumes_entregas for w in pesos_entregas \
                              if v <= volume_atual and w <= peso_atual and vetor_distancia[i] < delta]
        if len(possiveis_entregas) > 0:
            r = np.random.randint(len(possiveis_entregas))
            proxima_entrega = possiveis_entregas[r][0]
            volume_entrega = possiveis_entregas[r][1]
            peso_entrega = possiveis_entregas[r][2]
            
        while ponto_atual != proxima_entrega:
            trajetoria.append(proxima_entrega)
            ponto_atual = proxima_entrega
            volume_atual -= volume_entrega
            peso_atual -= peso_entrega
            
            i = pontos_entregas.index(ponto_atual)
            pontos_entregas.pop(i)
            volumes_entregas.pop(i)
            pesos_entregas.pop(i)
            
            if len(pontos_entregas) > 0:
                vetor_distancia = [np.linalg.norm(np.array(ponto_atual) - np.array(p)) for p in pontos_entregas]
                delta = min(vetor_distancia) + alpha*(max(vetor_distancia) - min(vetor_distancia))
                possiveis_entregas = [(pontos_entregas[i], v, w) for i in range(len(pontos_entregas)) \
                                      for v in volumes_entregas for w in pesos_entregas \
                                      if v <= volume_atual and w <= peso_atual and vetor_distancia[i] < delta]
                if len(possiveis_entregas) > 0:
                    r = np.random.randint(len(possiveis_entregas))
                    proxima_entrega = possiveis_entregas[r][0]
                    volume_entrega = possiveis_entregas[r][1]
                    peso_entrega = possiveis_entregas[r][2]

        #Descomentar para que a trajetoria seja um ciclo:
        #trajetoria.append(origem)
        custos_caixeiros.pop(caixeiro)
        volumes_caixeiros.pop(caixeiro)
        pesos_caixeiros.pop(caixeiro)
        if len(trajetoria) > 1:
            trajetorias[caixeiro] = trajetoria
    
    return trajetorias

## Algoritmo de Christofides-Serdyukov

O principal objetivo dessa heurística é melhorar o resultado obtido pela heurística construtiva.
Como esssa heurística também tem um componente não-determinístico ela será usada somente em 
conjunto com a heurística construtiva do vizinho mais próximo (que é determinística).

In [ ]:
# Copiado de https://github.com/Retsediv/ChristofidesAlgorithm
# Para o nosso problema, esse algoritmo não pode ser usado como uma
# heurística construtiva por causa das cargas.  Mas nós podemos usá-la como
# um primeiro passo para melhorar a solução dada pela heurística construtiva.
# Nesse caso, ela precisa ser aplicada na trajetória de cada caixeiro
# separadamente.  Além disso, é importante que ela seja aplicada antes das
# heurísticas de busca local: por um lado, para que ela não desfaça as
# melhorias feitas pelas buscas locais, e por outro lado, para que as buscas
# locais sejam capazes de melhorar o resultado dessa heurística.  Observe
# que essa heurística não é determinística, então é bom rodá-la várias vezes.

def christofides(data):
    origem = data[0]
    
    # build a graph
    G = matriz_distancia(data)
    #print("Graph: ", G)

    # build a minimum spanning tree
    MSTree = minimum_spanning_tree(G)
    #print("MSTree: ", MSTree)

    # find odd vertexes
    odd_vertexes = find_odd_vertexes(MSTree)
    #print("Odd vertexes in MSTree: ", odd_vertexes)

    # add minimum weight matching edges to MST
    minimum_weight_matching(MSTree, G, odd_vertexes)
    #print("Minimum weight matching: ", MSTree)

    # find an eulerian tour
    eulerian_tour = find_eulerian_tour(MSTree, G)
    #print("Eulerian tour: ", eulerian_tour)

    current = eulerian_tour[0]
    path = [data[current]]
    visited = [False] * len(eulerian_tour)
    visited[eulerian_tour[0]] = True
    length = 0

    for v in eulerian_tour:
        if not visited[v]:
            path.append(data[v])
            visited[v] = True

            length += G[current][v]
            current = v

    i = path.index(origem)
    path = path[i:] + path[:i]
    #Descomentar para que as trajetorias sejam ciclos
    #length +=G[current][eulerian_tour[0]]
    #path.append(data[eulerian_tour[0]])

    #print("Result path: ", path)
    #print("Result length of the path: ", length)
    return length, path


class UnionFind:
    def __init__(self):
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root

    def __iter__(self):
        return iter(self.parents)

    def union(self, *objects):
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r], r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest


def minimum_spanning_tree(G):
    tree = []
    subtrees = UnionFind()
    n = len(G)
    for W, u, v in sorted((G[u][v], u, v) for u in range(n) for v in range(u)):
        if subtrees[u] != subtrees[v]:
            tree.append((u, v, W))
            subtrees.union(u, v)

    return tree


def find_odd_vertexes(MST):
    tmp_g = {}
    vertexes = []
    for edge in MST:
        if edge[0] not in tmp_g:
            tmp_g[edge[0]] = 0

        if edge[1] not in tmp_g:
            tmp_g[edge[1]] = 0

        tmp_g[edge[0]] += 1
        tmp_g[edge[1]] += 1

    for vertex in tmp_g:
        if tmp_g[vertex] % 2 == 1:
            vertexes.append(vertex)

    return vertexes


def minimum_weight_matching(MST, G, odd_vert):
    import random
    random.shuffle(odd_vert)

    while odd_vert:
        v = odd_vert.pop()
        length = float("inf")
        u = 1
        closest = 0
        for u in odd_vert:
            if v != u and G[v][u] < length:
                length = G[v][u]
                closest = u

        MST.append((v, closest, length))
        odd_vert.remove(closest)


def find_eulerian_tour(MatchedMSTree, G):
    # find neigbours
    neighbours = {}
    for edge in MatchedMSTree:
        if edge[0] not in neighbours:
            neighbours[edge[0]] = []

        if edge[1] not in neighbours:
            neighbours[edge[1]] = []

        neighbours[edge[0]].append(edge[1])
        neighbours[edge[1]].append(edge[0])

    # finds the hamiltonian circuit
    start_vertex = MatchedMSTree[0][0]
    EP = [neighbours[start_vertex][0]]

    while len(MatchedMSTree) > 0:
        for i, v in enumerate(EP):
            if len(neighbours[v]) > 0:
                break

        while len(neighbours[v]) > 0:
            w = neighbours[v][0]

            remove_edge_from_matchedMST(MatchedMSTree, v, w)

            del neighbours[v][(neighbours[v].index(w))]
            del neighbours[w][(neighbours[w].index(v))]

            i += 1
            EP.insert(i, w)

            v = w

    return EP


def remove_edge_from_matchedMST(MatchedMST, v1, v2):

    for i, item in enumerate(MatchedMST):
        if (item[0] == v2 and item[1] == v1) or (item[0] == v1 and item[1] == v2):
            del MatchedMST[i]

    return MatchedMST

## Herísticas de busca local

### Swap inter-rotas

Essa heurística realiza modificações em um dicionário de trajetórias de caixeiros 
com o objetivo de diminuir a FO.  As modificações realizadas são: dados
dois caixeiros distintos, trocar um ponto de entrega na rota de um deles
por um ponto de entrega na rota do outro (ver figura a seguir).
![swap-inter-2.png](attachment:swap-inter-2.png)

In [ ]:
# As entradas dessa função são: um dicionário de números (que representa o 
# custo por km de cada caixeiro), um segundo dicionário de números (que
# representa o volume total que cada caixeiro consegue transportar), um 
# terceiro dicionário de números (que representa o peso total que cada 
# caixeiro consegue transportar), um dicionário de listas de arrays (cujas 
# entradas são os caixeiros e suas respectivas trajetórias), um dicionário
# de listas de números (cujas entradas são os caixeiros e os volumes de cada
# entrega contida na sua trajetória) e um segundo dicionário de listas de
# números (cujas entradas são os caixeiros e os pesos de cada entrega
# contida na sua trajetória).  A saída dessa função é um dicionário de
# listas de arrays, que representa as trajetórias (possivelmente) 
# modificadas de cada caixeiro.

def swap_inter(dict_num1, dict_num2, dict_num3, dict_listas_array, dict_listas_num1, dict_listas_num2):
    custos_caixeiros = copy.deepcopy(dict_num1)
    volumes_caixeiros = copy.deepcopy(dict_num2)
    pesos_caixeiros = copy.deepcopy(dict_num3)
    trajetorias = copy.deepcopy(dict_listas_array)
    volumes_entregas = copy.deepcopy(dict_listas_num1)
    pesos_entregas = copy.deepcopy(dict_listas_num2)
    
    fo_min = fo(trajetorias, custos_caixeiros)
    
    J = trajetorias.keys()
    for i in trajetorias.keys():
        J = [j for j in J if j != i]
        for j in J:
            for k in range(1, len(trajetorias[i])):
                for l in range(1, len(trajetorias[j])):
                    trajetorias[i][k], trajetorias[j][l] = trajetorias[j][l], trajetorias[i][k]
                    volumes_entregas[i][k], volumes_entregas[j][l] = volumes_entregas[j][l], volumes_entregas[i][k]
                    pesos_entregas[i][k], pesos_entregas[j][l] = pesos_entregas[j][l], pesos_entregas[i][k]
                    if ( sum(volumes_entregas[i]) <= volumes_caixeiros[i] and
                            sum(volumes_entregas[j]) <= volumes_caixeiros[j] and
                            sum(pesos_entregas[i]) <= pesos_caixeiros[i] and
                            sum(pesos_entregas[j]) <= pesos_caixeiros[j] and 
                            fo(trajetorias, custos_caixeiros) < fo_min ):
                        fo_min = fo(trajetorias, custos_caixeiros)
                    else:
                        trajetorias[i][k], trajetorias[j][l] = trajetorias[j][l], trajetorias[i][k]
                        volumes_entregas[i][k], volumes_entregas[j][l] = volumes_entregas[j][l], volumes_entregas[i][k]
                        pesos_entregas[i][k], pesos_entregas[j][l] = pesos_entregas[j][l], pesos_entregas[i][k]
    return trajetorias

### 2-opt

Essa heurística também realiza modificações em um dicionário de trajetórias de
caixeiros com o objetivo de diminuir a FO.  No entanto, as modificações
realizadas são: dados dois pontos de entrega na rota de um mesmo caixeiro,
inverter a ordem dos pontos da trajetória que estão contidos entre esses dois 
pontos (ver figura a seguir).
![swap-intra.png](attachment:swap-intra.png)

In [ ]:
# Formalmente, as entradas dessa função são as mesmas entradas da função 
# 'swap inter-rotas'.  No entanto, observe que várias dessas entradas não 
# serão efetivamente usadas nesta função.  No entanto, para aplicar a função
# 'buscas_locais' (abaixo), foi necessário padronizar as entradas de todas
# as buscas locais.
# As entradas dessa função que são efetivamente usadas são: dict_num1, um
# dicionário de números, cujas entradas representa os caixeiros e seus
# respectivos custos por km percorrido; e dict_listas_array, um dicionário
# de listas de arrays, cujas entradas representam os caixeiros e suas
# respectivas trajetórias.  A saída dessa função também é um dicionário de
# listas de arrays, cujas entradas representam os caixeiros e suas
# respectivas trajetórias (possivelmente modificadas). 

def dois_opt(dict_num1, dict_num2, dict_num3, dict_listas_array, dict_listas_num1, dict_listas_num2):
    trajetorias = copy.deepcopy(dict_listas_array)
    caixeiros = trajetorias.keys()

    for c in caixeiros:
        distancia = matriz_distancia(trajetorias[c])
        for i in range(1, len(trajetorias[c])-2):
            for j in range(i+1, len(trajetorias[c])):
                if distancia[i-1][i] + distancia[j-1][j] > distancia[i-1][j-1] + distancia[i][j]:
                    trajetorias[c] = [*trajetorias[c][0:i], *trajetorias[c][i:j][::-1], \
                                      *trajetorias[c][j:len(trajetorias[c])]]
    return trajetorias

A última função desta subseção combina as buscas locais contidas na lista 'funcoes' em um
procedimento que iteradamente busca por soluções cada vez melhores.

In [ ]:
# As entradas e saídas dessa função são exatamente as mesmas entradas e
# saídas de todas as funções de busca local definidas acima.  Para incluir
# ou excluir uma busca local desse procedimento, basta incluir ou excluir
# o nome da função correspondente no vetor 'funcoes'.


funcoes = [dois_opt, swap_inter]

def buscas_locais(dict_num1, dict_num2, dict_num3, dict_listas_array, dict_listas_num1, dict_listas_num2):
    custos_caixeiros = copy.deepcopy(dict_num1)
    volumes_caixeiros = copy.deepcopy(dict_num2)
    pesos_caixeiros = copy.deepcopy(dict_num3)
    trajetorias = copy.deepcopy(dict_listas_array)
    volumes_entregas = copy.deepcopy(dict_listas_num1)
    pesos_entregas = copy.deepcopy(dict_listas_num2)
    
    fo_min = fo(trajetorias, custos_caixeiros)
    
    i = 0
    k = 0
    imax = 100
    while k in range(len(funcoes)) and i < imax:
        f = funcoes[k]
        k += 1

        trajs = f(custos_caixeiros, volumes_caixeiros, pesos_caixeiros, trajetorias, volumes_entregas, pesos_entregas)
        cst = fo(trajs, custos_caixeiros)
        while cst < fo_min and i < imax:
            trajs = f(custos_caixeiros, volumes_caixeiros, pesos_caixeiros, trajs, volumes_entregas, pesos_entregas)
            cst = fo(trajetorias, custos_caixeiros)
            i += 1
            k = 0

        trajetorias = trajs
        fo_min = cst
        i += 1
    
    return trajetorias

# Meta-heurísticas

Nessa seção, nós definimos as funções que implementam as meta-heurísticas utilizadas para obter as soluções finais do nosso probema principal.

## Simulated annealing

In [ ]:
def simulated_annealing(dict_num1, dict_num2, dict_num3, dict_listas_array, dict_listas_num1, dict_listas_num2):
    custos_caixeiros = copy.deepcopy(dict_num1)
    volumes_caixeiros = copy.deepcopy(dict_num2)
    pesos_caixeiros = copy.deepcopy(dict_num3)
    trajetorias = copy.deepcopy(dict_listas_array)
    volumes_entregas = copy.deepcopy(dict_listas_num1)
    pesos_entregas = copy.deepcopy(dict_listas_num2)
    
    #t0, tmin, imax, alpha = 2*10**4, 0.01, 2000, 0.99
    t0, tmin, imax, alpha = 10**5, 10**(-2), 2000, 0.9

    t = t0
    i = 0

    fo_min = fo(trajetorias, custos_caixeiros)
    while t > tmin and i < imax:
        trajs = {c : christofides(trajetorias[c])[1] for c in trajetorias.keys()}
        trajs = buscas_locais(custos_caixeiros, volumes_caixeiros, pesos_caixeiros, trajs, volumes_entregas,\
                              pesos_entregas)
        delta = fo(trajs, custos_caixeiros) - fo_min
        if delta < 0 or random.random() < np.exp(-delta/t):
            trajetorias = trajs
            fo_min = fo(trajetorias, custos_caixeiros)
        i += 1
        t *= alpha
    print(f'FO: {fo(trajetorias, custos_caixeiros):0.2f}')
    return trajetorias

## Variable neighborhood descent (VNS)

### Com a heurística construtiva aleatória

In [ ]:
def vns_gr(array, dict_num1, dict_num2, dict_num3, lista_listas_array, lista_num1, lista_num2):
    origem = copy.deepcopy(array)
    custos_caixeiros = copy.deepcopy(dict_num1)
    volumes_caixeiros = copy.deepcopy(dict_num2)
    pesos_caixeiros = copy.deepcopy(dict_num3)
    pontos_entregas = copy.deepcopy(lista_listas_array)
    volumes_entregas = copy.deepcopy(lista_num1)
    pesos_entregas = copy.deepcopy(lista_num2)

    alpha = 0.3 # entre 0 e 1: 0 é completamete determinístico e 1 é completamente aleatório
    i = 0
    imax = 50
    fo_min = np.inf
    
    while i < imax:
        trajs = construtiva_aleatoria(alpha, origem, custos_caixeiros, volumes_caixeiros, pesos_caixeiros, pontos_entregas,\
                                      volumes_entregas, pesos_entregas)
        vents = lista2dict(pontos_entregas, volumes_entregas, trajs)
        pents = lista2dict(pontos_entregas, pesos_entregas, trajs)
        trajs = buscas_locais(custos_caixeiros, volumes_caixeiros, pesos_caixeiros, trajs, vents, pents)
        if fo(trajs, custos_caixeiros) < fo_min:
            trajetorias = trajs
            fo_min = fo(trajetorias, custos_caixeiros)
            i = 1
        else:
            i += 1
    print(f'FO: {fo(trajetorias, custos_caixeiros):0.2f}')
    return trajetorias

### Com a heurística construtiva do vizinho mais próximo

In [ ]:
def vns_nn(array, dict_num1, dict_num2, dict_num3, lista_listas_array, lista_num1, lista_num2):
    origem = copy.deepcopy(array)
    custos_caixeiros = copy.deepcopy(dict_num1)
    volumes_caixeiros = copy.deepcopy(dict_num2)
    pesos_caixeiros = copy.deepcopy(dict_num3)
    pontos_entregas = copy.deepcopy(lista_listas_array)
    volumes_entregas = copy.deepcopy(lista_num1)
    pesos_entregas = copy.deepcopy(lista_num2)

    i = 0
    imax = 50
    fo_min = np.inf
    
    while i < imax:
        trajs = construtiva_vizinho(origem, custos_caixeiros, volumes_caixeiros, pesos_caixeiros, pontos_entregas,\
                                    volumes_entregas, pesos_entregas)
        trajs = {c : christofides(trajs[c])[1] for c in trajs.keys()}
        vents = lista2dict(pontos_entregas, volumes_entregas, trajs)
        pents = lista2dict(pontos_entregas, pesos_entregas, trajs)
        trajs = buscas_locais(custos_caixeiros, volumes_caixeiros, pesos_caixeiros, trajs, vents, pents)
        if fo(trajs, custos_caixeiros) < fo_min:
            trajetorias = trajs
            fo_min = fo(trajetorias, custos_caixeiros)
            i = 1
        else:
            i += 1
    print(f'FO: {fo(trajetorias, custos_caixeiros):0.2f}')
    return trajetorias

## Greedy randomized adaptive search procedure (GRASP)

### Com a heurística construtiva aleatória

In [ ]:
def grasp(array, dict_num1, dict_num2, dict_num3, lista_vetores, lista_nums1, lista_nums2):
    origem = copy.deepcopy(array)
    custos_caixeiros = copy.deepcopy(dict_num1)
    volumes_caixeiros = copy.deepcopy(dict_num2)
    pesos_caixeiros = copy.deepcopy(dict_num3)
    pontos_entregas = copy.deepcopy(lista_vetores)
    volumes_entregas = copy.deepcopy(lista_nums1)
    pesos_entregas = copy.deepcopy(lista_nums2)

    imax = 50
    alpha = 0.45
    fo_min = np.inf

    for i in range(imax):
        trajs = construtiva_aleatoria(alpha, origem, custos_caixeiros, volumes_caixeiros, pesos_caixeiros, pontos_entregas,\
                                      volumes_entregas, pesos_entregas)
        vents = lista2dict(pontos_entregas, volumes_entregas, trajs)
        pents = lista2dict(pontos_entregas, pesos_entregas, trajs)
        trajs = buscas_locais(custos_caixeiros, volumes_caixeiros, pesos_caixeiros, trajs, vents, pents)
        if fo(trajs, custos_caixeiros) < fo_min:
            trajetorias = trajs
            fo_min = fo(trajetorias, custos_caixeiros)
    print(f'FO: {fo(trajetorias, custos_caixeiros):0.2f}')
    return trajetorias

### Com a heurística construtiva do vizinho mais próximo

In [ ]:
def grasp_nn(array, dict_num1, dict_num2, dict_num3, lista_vetores_R2, lista_num1, lista_num2):
    origem = copy.deepcopy(array)
    custos_caixeiros = copy.deepcopy(dict_num1)
    volumes_caixeiros = copy.deepcopy(dict_num2)
    pesos_caixeiros = copy.deepcopy(dict_num3)
    pontos_entregas = copy.deepcopy(lista_vetores_R2)
    volumes_entregas = copy.deepcopy(lista_num1)
    pesos_entregas = copy.deepcopy(lista_num2)
    
    imax = 50
    alpha = 0.45
    fo_min = np.inf
 
    for i in range(imax):
        trajs = construtiva_vizinho(origem, custos_caixeiros, volumes_caixeiros, pesos_caixeiros, pontos_entregas,\
                                    volumes_entregas, pesos_entregas)
        trajs = {c : christofides(trajs[c])[1] for c in trajs.keys()}
        vents = lista2dict(pontos_entregas, volumes_entregas, trajs)
        pents = lista2dict(pontos_entregas, pesos_entregas, trajs)
        trajs = buscas_locais(custos_caixeiros, volumes_caixeiros, pesos_caixeiros, trajs, vents, pents)
        if fo(trajs, custos_caixeiros) < fo_min:
            trajetorias = trajs
            fo_min = fo(trajetorias, custos_caixeiros)
    print(f'FO: {fo(trajetorias, custos_caixeiros):0.2f}')
    return trajetorias

# Testes e Resultados

In [ ]:
##########  TODOS OS TESTES ESTÃO CONCENTRADOS NESSA CÉLULA  #########


# Simulated annealing
print('* Simulated annealing *')
start = time.time()
trajs = construtiva_vizinho(origem, custos_caixeiros, volumes_caixeiros, pesos_caixeiros, pontos_entregas,\
                            volumes_entregas, pesos_entregas)
vents = lista2dict(pontos_entregas, volumes_entregas, trajs)
pents = lista2dict(pontos_entregas, pesos_entregas, trajs)
trajs = simulated_annealing(custos_caixeiros, volumes_caixeiros, pesos_caixeiros, trajs, vents, pents)
end = time.time()
print(f'Tempo de execução: {end-start:0.3f} segundos')
ver_rotas(origem, trajs)


# VNS com a heurística construtiva do GRASP
start = time.time()
print('* VNS / grasp *')
trajs = vns_gr(origem, custos_caixeiros, volumes_caixeiros, pesos_caixeiros, pontos_entregas, volumes_entregas,\
               pesos_entregas)
end = time.time()
#print(f'FO: {fo(trajs, custos_caixeiros):0.3f}')
print(f'Tempo de execução: {end-start:0.3f} segundos')
ver_rotas(origem, trajs)


# VNS com a heurística construtiva do vizinho mais próximo
start = time.time()
print('* VNS / nn *')
trajs = vns_nn(origem, custos_caixeiros, volumes_caixeiros, pesos_caixeiros, pontos_entregas, volumes_entregas,\
               pesos_entregas)
end = time.time()
#print(f'FO: {fo(trajs, custos_caixeiros):0.3f}')
print(f'Tempo de execução: {end-start:0.3f} segundos')
ver_rotas(origem, trajs)


# GRASP 
start = time.time()
print('* GRASP *')
trajs = grasp(origem, custos_caixeiros, volumes_caixeiros, pesos_caixeiros, pontos_entregas, volumes_entregas,\
               pesos_entregas)
end = time.time()
#print(f'FO: {fo(trajs, custos_caixeiros):0.3f}')
print(f'Tempo de execução: {end-start:0.3f} segundos')
ver_rotas(origem, trajs)


# GRASP com a heurística construtiva do vizinho mais próximo
start = time.time()
print('* GRASP / nn *')
trajs = grasp_nn(origem, custos_caixeiros, volumes_caixeiros, pesos_caixeiros, pontos_entregas, volumes_entregas,\
                 pesos_entregas)
end = time.time()
#print(f'FO: {fo(trajs, custos_caixeiros):0.3f}')
print(f'Tempo de execução: {end-start:0.3f} segundos')
ver_rotas(origem, trajs)